In [2]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
spam_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data Science/Intelipaat/Lecture/NLP/NLP-20221204T164251Z-001/NLP/spam.csv", 
                        encoding="latin-1")

spam_data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Target,Text,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [3]:
spam_data.Target.value_counts()
spam_data = spam_data[["Target", "Text"]]


In [4]:

### Text Similarty
import spacy
nlp = spacy.load("en_core_web_sm")

def clean_doc(sent):
    doc = nlp(sent)
    lemma_out = [token.lemma_ for token in doc]
    lemma_out = " ".join(lemma_out)
    return lemma_out

spam_data["Text_clean"] = spam_data.Text.apply(lambda x: clean_doc(x))
spam_data["Text_clean"]

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
<ipython-input-4-76b7a31d75f9>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spam_data["Text_clean"] = spam_data.Text.apply(lambda x: clean_doc(x))


0       go until jurong point , crazy .. available onl...
1                           ok lar ... joke wif u oni ...
2       free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor ... U c already then sa...
4       Nah I do not think he go to usf , he live arou...
                              ...                        
5567    this be the 2nd time we have try 2 contact u. ...
5568                 will Ì _ b go to esplanade fr home ?
5569    pity , * be in mood for that . so ... any othe...
5570    the guy do some bitching but I act like I woul...
5571                          Rofl . its true to its name
Name: Text_clean, Length: 5572, dtype: object

In [5]:
def count_digits(text):
    num_digits = 0
    for char in text:
        if char.isdigit():
            num_digits+=1
    return num_digits

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vec = CountVectorizer(stop_words="english", min_df = 0.01)
# vec = TfidfVectorizer(stop_words="english", min_df = 0.001)
vec = vec.fit(spam_data["Text_clean"])
vectorized_data = vec.transform(spam_data["Text_clean"])

vectorized_data = vectorized_data.toarray()
vectorized_data = pd.DataFrame(vectorized_data)

vectorized_data["num_digits"] = spam_data.Text.apply(lambda x: count_digits(x))

In [6]:

vectorized_data["num_digits"]


0        0
1        0
2       25
3        0
4        0
        ..
5567    21
5568     0
5569     0
5570     0
5571     0
Name: num_digits, Length: 5572, dtype: int64

In [7]:
## Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vectorized_data, spam_data["Target"], test_size=0.3)

In [8]:
## Model Building
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


RandomForestClassifier()

In [9]:
preds = clf.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1456
        spam       0.98      0.91      0.94       216

    accuracy                           0.99      1672
   macro avg       0.98      0.95      0.97      1672
weighted avg       0.99      0.99      0.99      1672



/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
